In [1]:
#일부만 할당된 GPU 메모리를 실행하는 동안 필요한 만큼 늘릴 수 있도록 설정함
# 모델 학습 시 불필요한 출력을 끄도록 함 (warning)

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
config =tf.ConfigProto()
config.gpu_options.allow_growth= True
session = tf.Session(config=config)

In [2]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np


config =tf.ConfigProto()
config.gpu_options.allow_growth= True
session = tf.Session(config=config)

# 모델 학습 시 불필요한 출력을 끄도록 함 (warning)

tf.logging.set_verbosity(tf.logging.ERROR)

In [3]:
#밴딧 정의 : 4개의 손잡이를 가지는 밴딧. pullBandit 함수는 평균 값 0의 정규 분포로부터 랜덤한 숫자를 생성. 
#밴딧의 수가 작을수록 +의 보상이 돌아올 가능성이 높습니다. 
#우리는 에이전트가 언제나 +의 보상을 가져올 손잡이를 선택하도록 학습하도록 설계
#밴딧의 손잡이 목록을 작성
#현재 손잡이 4(인덱스 3)이 가장 자주 양의 보상을 제공하도록 설정

bandit_arms = [0.2, 0, -0.2, -2] # 결국 -값을 크게 해줄 수록당길 확률이 높아짐
num_arms = len(bandit_arms)

def pullBandit(bandit): #어떤 레버를 당길 것인지 랜덤값을 구한다.
    #표준정규분포의 랜덤 넘버를 만들어서 그 값보다 작으면 1을 줌 0에서 멀어질수록 보상을 받을 확률이 큼 -> 레버 4가 확률이 가장 높게 만들려고.
    result = np.random.randn(1)
    if result > bandit :
        return 1 #양의 보상을 반환
    
    else:
        return -1 #음의 보상을 반환

In [4]:
#에이전트 아래의 코드는 간단한 신경망을 구축합니다. 
#이 신경망은 각각의 밴딧 손잡이에 대한 일련의 값들로 구성되어 있다.
#각각의 값은 밴딧의 선택에 의해 반환되는 값의 추정치
#우리는 정책 경사 방법을 이용하여 선택된 액션에 대한 값을 반환받은 보상 쪽으로 이동시켜 가는 에이전트를 업데이트시켜 나갑니다.
# tf.Variable : A variable maintains state in the graph across calls to run(). You add a variable to the graph by constructing an instance of the class Variable. requires an initial value for the variable, which can be a Tensor of any type and shape.
#tf.slice (input_, begin, size, name=None) : 텐서의 begin(각 차원에서의 offset)에서 size(텐서)만큼 을 추출, size[[i] input에서 추출할 i번째 차원의 원소


tf.reset_default_graph() #clear 시킴

# 네트워크의 피드-포워드 부분을 구축한다.
weights = tf.Variable(tf.ones([num_arms])) #초기값이 1로 주어지고 add a viable-run()
output = tf.nn.softmax(weights) #나오는 값?

print(output)

#학습과정을 구축한다 
#보상과 선택된 액션을 네트워크에 피드해 줌으로써 비용을 계산하고 비용을 이용해 네트워크를 업데이트
reward_holder = tf.placeholder(shape=[1], dtype=tf.float32) #?placeholder
action_holder = tf.placeholder(shape=[1], dtype=tf.int32) #?
tf.slice # 시작을 알림
responsible_output = tf.slice(output,action_holder,[1]) #responsible_output? : softmax로 받은 output을 placeholder shape[1]에서 size[1]만큼 추출
loss = -(tf.log(responsible_output)*reward_holder) #reward_holder?
optimizer= tf.train.AdamOptimizer(learning_rate=1e-3)
update = optimizer.minimize(loss)


Tensor("Softmax:0", shape=(4,), dtype=float32)


In [5]:
#에이전트 학습시키기
#이제 우리는 우리의 한경 내에서 액션을 취함으로써 에이전트를 학습시키고 보상을 받게 됩니다.
#보상과 앳견을 이용함으로써 우리는 시간의 경과에 따라 최고의 보상을 받게 될 액션을 보다 자주 선택하기 위해
#네트워크를 어떻게 업데이트시켜 나갈 지에 대해 알 수 있게 됩니다.
#tf.global_variables_initializer() : 모델의 다른 연산을 실행하기 전에 반드시 실행, 가장 쉬운 방법1.은 모든 변수를 초기화하는 연산을 모델 사용전에 실행하는 것. 2. 체크포인트 파일에서 변수 값을 복원할 수 있음.
# np.random.choice : 이미 있는 데이터 집합에서 일부를 무작위로 선택하는 것을 샘플링(sampling)이라고 한다. 샘플링에는 choice 명령을 사
# a : 배열이면 원래의 데이터, 정수이면 arange(a) 명령으로 데이터 생성
# size : 정수. 샘플 숫자
# replace : 불리언. True이면 한번 선택한 데이터를 다시 선택 가능
# p : 배열. 각 데이터가 선택될 수 있는 확률

total_episodes = 1000 #학습시킬 전체 에피소드 횟수 설정
total_reward = np.zeros(num_arms)

init = tf.global_variables_initializer() #변수 초기화는 

#텐서플로우 그래프를 론칭한다.
with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:
        #볼츠만 분포에 따라 액션 선택
        actions = sess.run(output)
        a = np.random.choice(actions, p = actions) #actions의 확률에 따라 actions중하나를 뽑는다.
        action = np.argmax(actions == a) # actions ==a 인 True 1의 인덱스
        
        reward = pullBandit(bandit_arms[action]) # 밴딧 손잡이 중 하나를 선택함으로써 보상받음
        
        #네트워크를 업데이트한다
        a,resp,ww = sess.run([update, responsible_output, weights], feed_dict={reward_holder:[reward], action_holder:[action]})
        
        #보상 층계 업데이트
        total_reward[action] += reward
        if i % 50 == 0:
            print("Running reward for the " + str(num_arms) + " arms of the bandit: " + str(total_reward))
        i+=1
print("\nThe agent thinks arm " + str(np.argmax(ww)+1) + " is the most promising....")
if np.argmax(ww) == np.argmax(-np.array(bandit_arms)):
    print("...and it was right!")
else:
    print("...and it was wrong!")

Running reward for the 4 arms of the bandit: [-1.  0.  0.  0.]
Running reward for the 4 arms of the bandit: [-1.  0. -3. 11.]
Running reward for the 4 arms of the bandit: [-5. -6. -3. 23.]
Running reward for the 4 arms of the bandit: [-15.  -9.  -4.  33.]
Running reward for the 4 arms of the bandit: [-16. -15.  -7.  43.]
Running reward for the 4 arms of the bandit: [-17. -21.  -2.  55.]
Running reward for the 4 arms of the bandit: [-15. -26.   4.  74.]
Running reward for the 4 arms of the bandit: [-17. -26.  12.  92.]
Running reward for the 4 arms of the bandit: [-16. -29.  14. 104.]
Running reward for the 4 arms of the bandit: [-22. -33.  15. 119.]
Running reward for the 4 arms of the bandit: [-21. -34.  14. 136.]
Running reward for the 4 arms of the bandit: [-23. -30.  18. 156.]
Running reward for the 4 arms of the bandit: [-21. -29.  19. 168.]
Running reward for the 4 arms of the bandit: [-22. -27.  19. 179.]
Running reward for the 4 arms of the bandit: [-30. -30.  21. 192.]
Running

In [14]:
"""
https://towardsdatascience.com/reinforcement-learning-rl-101-with-python-e1aa0d37d43b
Monte Carlo for 4*4 Gridworld Problem
2019.07.20
"""

import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
%pylab inline
import random

#===============================#
## First-visit Monte Carlo
#===============================#

# parameters
gamma = 0.6 # discounting rate
rewardSize = -1
gridSize = 4
terminationStates = [[0,0], [gridSize-1, gridSize-1]]
actions = [[-1, 0], [1, 0], [0, 1], [0, -1]]  # up, down, left, right
numIterations = 10000

# initialization
V = np.zeros((gridSize, gridSize)) # gridSize에 맞춰서 0으로 입력해줌
returns = {(i, j):list() for i in range(gridSize) for j in range(gridSize)} #dictionary with keys and values 
deltas = {(i, j):list() for i in range(gridSize) for j in range(gridSize)}  #dictionary with keys and values 
states = [[i, j] for i in range(gridSize) for j in range(gridSize)]

# utils
def generateEpisode():
    initState = random.choice(states[1:-1])
    episode = []
    while True:
        if list(initState) in terminationStates:# initState가 도착지에 만들어지면
            return episode
        action = random.choice(actions) 
        finalState=np.array(initState)+np.array(action)
        if -1 in list(finalState) or gridSize in list(finalState): # lsist(finalState)의 -1/gridSize 벽에 부딪힌경우 - 제자리로 돌아감
            finalState = initState
        episode.append([list(initState), action, rewardSize, list(finalState)]) # action기록
        initState = finalState

for it in tqdm(range(numIterations)):  #tqdm 상태바 출력 원하는 횟수만큼
    episode = generateEpisode()
    G = 0
    for i, step in enumerate(episode[::-1]): # [::-1]순서를 거꾸로 해줌 - goal로 들어간 제일 마지막 거부터 감마를 곱해서 계산 하고 그걸 입력으로 받아 계산하고 그래서 순서를 거꾸로해줌 
        G = gamma*G + step[2] # step[2]=리워드 : 할인율 적용해서 리워드 계산
        if step[0] not in [x[0] for x in episode[::-1][len(episode)-i:]]: # step[0]=list(initState) 거꾸로[len(episode)-0:]: len-i번쨰부터 뒤로인데 i=0일때는 인덱스넘버가 길이(len)인 것은 없으니까 보정
            idx = (step[0][0], step[0][1]) #step[0]=list(initState)[0],[1] 0,1 번째
            returns[idx].append(G)
            newValue = np.average(returns[idx])
            deltas[idx[0], idx[1]].append(np.abs(V[idx[0], idx[1]]-newValue))
            V[idx[0], idx[1]] = newValue

print("Value Table\n", V)
#V table에서 

# using gamma = 1
plt.figure(figsize=(20,10))
all_series = [list(x)[:50] for x in deltas.values()]
for series in all_series:
    plt.plot(series)

SyntaxError: invalid syntax (<ipython-input-14-8f306ea179cd>, line 59)

In [ ]:
pip install gym

# -*- coding: utf-8 -*-
"""
Example Design: Self-Driving Cab
Solving the Cab Problem using Q Learning
https://gym.openai.com/envs/Taxi-v2/
"""
"""
    The Taxi Problem
    from "Hierarchical Reinforcement Learning with the MAXQ Value Function Decomposition"
    by Tom Dietterich

    Description:
    There are four designated locations in the grid world indicated by R(ed), B(lue), G(reen), and Y(ellow). 
    When the episode starts, the taxi starts off at a random square and the passenger is at a random location. 
    The taxi drive to the passenger's location, pick up the passenger, 
    drive to the passenger's destination (another one of the four specified locations), 
    and then drop off the passenger. Once the passenger is dropped off, the episode ends.

    Observations: 
    There are 500 discrete states 
    since there are 25 taxi positions, 5 possible locations 
    of the passenger (including the case when the passenger is the taxi), and 4 destination locations. 
    
    Actions: 
    There are 6 discrete deterministic actions:
    - 0: move south
    - 1: move north
    - 2: move east 
    - 3: move west 
    - 4: pickup passenger
    - 5: dropoff passenger
    
    Rewards: 
    There is a reward of -1 for each action and an additional reward of +20 for delievering the passenger. 
    There is a reward of -10 for executing actions "pickup" and "dropoff" illegally.
    

    Rendering:
    - blue: passenger
    - magenta: destination
    - yellow: empty taxi
    - green: full taxi
    - other letters (R, G, B and Y): locations for passengers and destinations

    actions:
    - 0: south
    - 1: north
    - 2: east
    - 3: west
    - 4: pickup
    - 5: dropoff

    state space is represented by:
        (taxi_row, taxi_col, passenger_location, destination)
"""

In [10]:
#import numpy as np
import random
import gym

env = gym.make('Taxi-v2')

#​env.render()

q={}
for s in range(env.observation_space.n): #observation_space 
    for a in range(env.action_space.n): # action
        q[(s,a)] =0.0

def update_q_table(prev_state, action, reward, nextstate, alpha, gamma):
    qa = max([q[(nextstate, a)] for a in range(env.action_space.n)])
    q[(prev_state, action)] += alpha* (reward + gamma * qa - q[(prev_state, action)])
    
def epsilon_greedy_policy(state, epsilon):
    if random.uniform(0,1) < epsilon: # epsilon 보다 작은 경우 explpration, 큰 경우 경험을 이용하는 exploitation
        return env.action_space.sample()
    else:
        return max(list(range(env.action_space.n)), key= lambda x: q[(state, x)]) # max() key= 반환할 모양
    

alpha = 0.4
gamma = 0.999
epsilon = 0.017

for i in range(80):
    r=0
    
    prev_state =env.reset()
    
    while True:
        env.render()

        # in each state, we select the action by epsilon-greedy policy
        action = epsilon_greedy_policy(prev_state, epsilon)

        #than we perform the action and move to the next state, and receive the reward
        nextstate, reward, done, _ = env.step(action) #step이란 함수가 action하고 move하고 reward 받는 건가

        #Next we update the Q value using our update_q_table function
        #which updates the Q value by Q learning update rule

        update_q_table(prev_state, action, reward, nextstate, alpha, gamma)

        #Finally we update the previous state as next state
        prev_state = nextstate

        #store all the rewards obtained
        r+= reward

        # we will break the loop, if we are at the terminal state of the episode
        if done:
            break
            
    print("total reward:", r)

env.close()
    

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | 

  (Pickup)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | 

  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | 

  (East)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | 

...

  (East)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |

  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | 

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
|_: : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| :_: : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : :_: : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : : : : |
| : :_: : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | :_| : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| :_|B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| :_|B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | :_| : |
|Y|

  (East)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : 

  (East)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : 